# 11.4 时区处理

时间序列处理工作中最让人不爽的就是对时区的处理。许多人都选择以协调世界时（UTC，它是格林尼治标准时间（Greenwich Mean Time）的接替者，目前已经是国际标准了）来处理时间序列。时区是以UTC偏移量的形式表示的。例如，夏令时期间，纽约比UTC慢4小时，而在全年其他时间则比UTC慢5小时。

在Python中，时区信息来自第三方库pytz，它使Python可以使用Olson数据库（汇编了世界时区信息）。这对历史数据非常重要，这是因为由于各地政府的各种突发奇想，夏令时转变日期（甚至UTC偏移量）已经发生过多次改变了。就拿美国来说，DST转变时间自1900年以来就改变过多次！

有关pytz库的更多信息，请查阅其文档。就本书而言，由于pandas包装了pytz的功能，因此你可以不用记忆其API，只要记得时区的名称即可。时区名可以在shell中看到，也可以通过文档查看：

In [14]:
from datetime import datetime

import pandas as pd
import numpy as np

In [2]:
import pytz


pytz.common_timezones

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Tripoli',
 'Africa/Tunis',
 'Africa/Wi

In [4]:
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

要从pytz中获取时区对象，使用pytz.timezone即可：

In [12]:
tz = pytz.timezone('Asia/Shanghai')

tz

<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>

pandas中的方法既可以接受时区名也可以接受这些对象。

# 时区本地化和转换

默认情况下，pandas中的时间序列是单纯（naive）的时区。看看下面这个时间序列：

In [19]:
rng = pd.date_range('3/1/2023 9:30', 
                    periods=6, 
                    freq='D'
                   )

rng

DatetimeIndex(['2023-03-01 09:30:00', '2023-03-02 09:30:00',
               '2023-03-03 09:30:00', '2023-03-04 09:30:00',
               '2023-03-05 09:30:00', '2023-03-06 09:30:00'],
              dtype='datetime64[ns]', freq='D')

In [21]:
ts = pd.Series(np.random.randn(len(rng)), 
               index=rng
              )

ts

2023-03-01 09:30:00   -1.193163
2023-03-02 09:30:00    0.747113
2023-03-03 09:30:00   -0.734443
2023-03-04 09:30:00   -0.678025
2023-03-05 09:30:00    0.109747
2023-03-06 09:30:00    1.478378
Freq: D, dtype: float64

其索引的tz字段为None：

In [18]:
print(ts.index.tz)

None


可以用时区集生成日期范围：

In [26]:
pd.date_range('3/9/2023 9:30', 
              periods=10, 
              freq='D', 
              tz='UTC'
             )

DatetimeIndex(['2023-03-09 09:30:00+00:00', '2023-03-10 09:30:00+00:00',
               '2023-03-11 09:30:00+00:00', '2023-03-12 09:30:00+00:00',
               '2023-03-13 09:30:00+00:00', '2023-03-14 09:30:00+00:00',
               '2023-03-15 09:30:00+00:00', '2023-03-16 09:30:00+00:00',
               '2023-03-17 09:30:00+00:00', '2023-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

从单纯到本地化的转换是通过tz_localize方法处理的：

In [27]:
ts

2023-03-01 09:30:00   -1.193163
2023-03-02 09:30:00    0.747113
2023-03-03 09:30:00   -0.734443
2023-03-04 09:30:00   -0.678025
2023-03-05 09:30:00    0.109747
2023-03-06 09:30:00    1.478378
Freq: D, dtype: float64

In [29]:
ts_utc = ts.tz_localize('UTC')

ts_utc

2023-03-01 09:30:00+00:00   -1.193163
2023-03-02 09:30:00+00:00    0.747113
2023-03-03 09:30:00+00:00   -0.734443
2023-03-04 09:30:00+00:00   -0.678025
2023-03-05 09:30:00+00:00    0.109747
2023-03-06 09:30:00+00:00    1.478378
Freq: D, dtype: float64

In [30]:
ts_utc.index

DatetimeIndex(['2023-03-01 09:30:00+00:00', '2023-03-02 09:30:00+00:00',
               '2023-03-03 09:30:00+00:00', '2023-03-04 09:30:00+00:00',
               '2023-03-05 09:30:00+00:00', '2023-03-06 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

一旦时间序列被本地化到某个特定时区，就可以用tz_convert将其转换到别的时区了：

In [31]:
ts_utc.tz_convert('Asia/Shanghai')

2023-03-01 17:30:00+08:00   -1.193163
2023-03-02 17:30:00+08:00    0.747113
2023-03-03 17:30:00+08:00   -0.734443
2023-03-04 17:30:00+08:00   -0.678025
2023-03-05 17:30:00+08:00    0.109747
2023-03-06 17:30:00+08:00    1.478378
Freq: D, dtype: float64

对于上面这种时间序列（它跨越了美国东部时区的夏令时转变期），我们可以将其本地化到EST，然后转换为UTC或柏林时间：

In [33]:
ts_eastern = ts.tz_localize('Asia/Shanghai')

ts_eastern.tz_convert('UTC')

2023-03-01 01:30:00+00:00   -1.193163
2023-03-02 01:30:00+00:00    0.747113
2023-03-03 01:30:00+00:00   -0.734443
2023-03-04 01:30:00+00:00   -0.678025
2023-03-05 01:30:00+00:00    0.109747
2023-03-06 01:30:00+00:00    1.478378
dtype: float64

In [34]:
ts_eastern.tz_convert('Europe/Berlin')

2023-03-01 02:30:00+01:00   -1.193163
2023-03-02 02:30:00+01:00    0.747113
2023-03-03 02:30:00+01:00   -0.734443
2023-03-04 02:30:00+01:00   -0.678025
2023-03-05 02:30:00+01:00    0.109747
2023-03-06 02:30:00+01:00    1.478378
dtype: float64

tz_localize和tz_convert也是DatetimeIndex的实例方法：

In [35]:
ts.index.tz_localize('Asia/Shanghai')

DatetimeIndex(['2023-03-01 09:30:00+08:00', '2023-03-02 09:30:00+08:00',
               '2023-03-03 09:30:00+08:00', '2023-03-04 09:30:00+08:00',
               '2023-03-05 09:30:00+08:00', '2023-03-06 09:30:00+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', freq=None)

> 注意：对单纯时间戳的本地化操作还会检查夏令时转变期附近容易混淆或不存在的时间。

## 操作时区意识型Timestamp对象

跟时间序列和日期范围差不多，独立的Timestamp对象也能被从单纯型（naive）本地化为时区意识型（time zone-aware），并从一个时区转换到另一个时区：

In [39]:
stamp = pd.Timestamp('2023-03-12 04:00')
print(stamp)


stamp_utc = stamp.tz_localize('utc')
stamp_utc

2023-03-12 04:00:00


Timestamp('2023-03-12 04:00:00+0000', tz='UTC')

In [40]:
stamp_utc.tz_convert('Asia/Shanghai')

Timestamp('2023-03-12 12:00:00+0800', tz='Asia/Shanghai')

在创建Timestamp时，还可以传入一个时区信息：

In [42]:
stamp_moscow = pd.Timestamp('2023-03-12 04:00', tz='Europe/Moscow')

stamp_moscow

Timestamp('2023-03-12 04:00:00+0300', tz='Europe/Moscow')

时区意识型Timestamp对象在内部保存了一个UTC时间戳值（自UNIX纪元（1970年1月1日）算起的纳秒数）。这个UTC值在时区转换过程中是不会发生变化的：

In [43]:
stamp_utc.value

1678593600000000000

In [44]:
stamp_utc.tz_convert('America/New_York').value

1678593600000000000

当使用pandas的DateOffset对象执行时间算术运算时，运算过程会自动关注是否存在夏令时转变期。这里，我们创建了在DST转变之前的时间戳。首先，来看夏令时转变前的30分钟：

In [45]:
from pandas.tseries.offsets import Hour

In [47]:
stamp = pd.Timestamp('2023-03-12 01:30', tz='US/Eastern')

stamp

Timestamp('2023-03-12 01:30:00-0500', tz='US/Eastern')

In [48]:
stamp + Hour()

Timestamp('2023-03-12 03:30:00-0400', tz='US/Eastern')

然后，夏令时转变前90分钟：

In [50]:
stamp = pd.Timestamp('2023-11-04 00:30', tz='US/Eastern')

stamp

Timestamp('2023-11-04 00:30:00-0400', tz='US/Eastern')

In [51]:
stamp + 2 * Hour()

Timestamp('2023-11-04 02:30:00-0400', tz='US/Eastern')

## 不同时区之间的运算

如果两个时间序列的时区不同，在将它们合并到一起时，最终结果就会是UTC。由于时间戳其实是以UTC存储的，所以这是一个很简单的运算，并不需要发生任何转换：

In [52]:
rng = pd.date_range('3/7/2023 9:30', 
                    periods=10, 
                    freq='B'
                   )

rng

DatetimeIndex(['2023-03-07 09:30:00', '2023-03-08 09:30:00',
               '2023-03-09 09:30:00', '2023-03-10 09:30:00',
               '2023-03-13 09:30:00', '2023-03-14 09:30:00',
               '2023-03-15 09:30:00', '2023-03-16 09:30:00',
               '2023-03-17 09:30:00', '2023-03-20 09:30:00'],
              dtype='datetime64[ns]', freq='B')

In [53]:
ts = pd.Series(np.random.randn(len(rng)), 
               index=rng
              )

ts

2023-03-07 09:30:00   -0.854463
2023-03-08 09:30:00    1.065184
2023-03-09 09:30:00    0.280491
2023-03-10 09:30:00   -0.291366
2023-03-13 09:30:00    0.428622
2023-03-14 09:30:00    1.848413
2023-03-15 09:30:00   -0.510597
2023-03-16 09:30:00    0.327308
2023-03-17 09:30:00    1.874046
2023-03-20 09:30:00    0.273881
Freq: B, dtype: float64

In [54]:
ts1 = ts[:7].tz_localize('Europe/London')
ts2 = ts1[2:].tz_convert('Europe/Moscow')


result = ts1 + ts2
result

2023-03-07 09:30:00+00:00         NaN
2023-03-08 09:30:00+00:00         NaN
2023-03-09 09:30:00+00:00    0.560983
2023-03-10 09:30:00+00:00   -0.582733
2023-03-13 09:30:00+00:00    0.857245
2023-03-14 09:30:00+00:00    3.696826
2023-03-15 09:30:00+00:00   -1.021195
dtype: float64

In [56]:
result.indexs

DatetimeIndex(['2023-03-07 09:30:00+00:00', '2023-03-08 09:30:00+00:00',
               '2023-03-09 09:30:00+00:00', '2023-03-10 09:30:00+00:00',
               '2023-03-13 09:30:00+00:00', '2023-03-14 09:30:00+00:00',
               '2023-03-15 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)